# Lightning Reader on Your Phone!

Don't have time to read through a text passage to find what the answer is? We got you!

With this telegram bot - equipped with AI capabilities - you will be able to instantly answer questions without reading the text passage.

# AI for QnA

In [ ]:
!pip install tensorflow==2.13.0
!pip install tflite-support==0.4.4

In [ ]:
from tflite_support.task import text
import requests

"""
Returns a TFLite BERT Classifier using the pretrained model for answering questions based on context.
"""
def get_model():
  model_url = "https://tfhub.dev/tensorflow/lite-model/mobilebert/1/metadata/1?lite-format=tflite"
  r = requests.get(model_url)
  with open('mobilebert.tflite','wb') as f:
    f.write(r.content)

  answerer = text.BertQuestionAnswerer.create_from_file("mobilebert.tflite")
  return answerer

"""
Using the given answerer and context, reply with the answer for the question that has the highest
possibility.
"""
def answer_based_on_context(answerer, context, question):
  answer_list = answerer.answer(context, question)
  answer = sorted(answer_list.answers, reverse=True, key=lambda x: x.pos.logit)[0]

  return answer.text

In [ ]:
answerer = get_model()
answer_based_on_context(answerer, "Bob has 3 apples.", "How many apples do Bob have?")

# Creating Telegram Chatbot
First, search BotFather in Telegram, and request for a new bot by typing /newbot.

You will be prompted to give your bot a username, and you will get the bot token.

Here, we will create a telegram chatbot that replies to your every question based on the context.

In [ ]:
!pip install python-telegram-bot

In [ ]:
from telegram import Update
from telegram.ext import filters, ApplicationBuilder, ContextTypes, MessageHandler, CommandHandler
import nest_asyncio

text_context = ""

async def answer_question_with_context(update: Update, context: ContextTypes.DEFAULT_TYPE):
  if (text_context == None or text_context == ""):
    await context.bot.send_message(chat_id=update.effective_chat.id, text="Context is not set!")

  reply = answer_based_on_context(answerer, text_context, update.message.text)
  await context.bot.send_message(chat_id=update.effective_chat.id, text=reply)

async def set_context(update: Update, context: ContextTypes.DEFAULT_TYPE):
  global text_context
  text_context = update.message.text

  await context.bot.send_message(chat_id=update.effective_chat.id, text="Context Updated!")

if __name__ == '__main__':
  application = ApplicationBuilder().token('').build()

  context_handler = CommandHandler('context', set_context)
  application.add_handler(context_handler)

  answer_handler = MessageHandler(filters.TEXT & (~filters.COMMAND), answer_question_with_context)
  application.add_handler(answer_handler)

  nest_asyncio.apply()
  application.run_polling()